In [2]:
import os
import openai

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
openai.api_key = OPENAI_API_KEY

In [7]:
messages = [{"role": "system", "content": "Hello World"}]
response = openai.ChatCompletion.create(
    model="gpt-4o-mini-2024-07-18",
    messages=messages,
    n=1,
    stop=None,
)
response.choices[0].message.content

'Hello! How can I assist you today?'